In [18]:
from sympy import init_session
import numpy as np
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
init_session()
from tqdm import tqdm
from IPython.display import display
from itertools import product

IPython console for SymPy 1.6.2 (Python 3.7.3-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.6.2/



In [1]:
from sympy import *

In [20]:
def get_factor(number: int):
    factors = list(factorint(number).keys())
    factors.append(number)
    factors.append(1)
    factors.append(-1)
    factors.extend([-x for x in factors.copy()])
    return np.unique(np.array(factors))

In [31]:
x, y, z, t = symbols('x y z t')
# исходный полином
X = Poly(x ** 5 - x ** 4 - 2 * x ** 3 - 8 * x ** 2 + 6 * x - 1)
display(X)
n = degree(X) // 2
# вектор точек, в которых необходимо
R = np.array(range(0,n+1))
# вектор делителей
factors = list()
# раскладываем множество делителей X (i), где i это число это от 0 до n
for i in tqdm(range(0, n+1)):
    t = X(i)
    factors.append(get_factor(t))
    factors[i].sort()
factors_len = 1
# считаем число вариантов которые необходимо перебрать
for f in factors:
    factors_len = f.shape[0] * factors_len
# и делим его на 2 чтобы не перебирать лишние комбинации
factors_len = factors_len // 2
print(factors_len)
# выполняем прямое произведение между членами вектора факторов
ys = product(*factors)
# и начинаем перебор
for i, y_i in tqdm(enumerate(ys), total=factors_len):
    # интерполяция по Лагранжу
    k = lagrange(R, y_i)
    buf_poly = Poly(k.c, x)
    if degree(buf_poly) > 0:
        # если исходный полином делиться на интерполяционный без остатка то выводим его
        quo, rem =  X.div(buf_poly)
        # print(rem)
        if rem == 0:
            display("Интерполяционный полином:")
            display(buf_poly)
            display("Результат деления исходного на интерполяционный:")
            display(quo)
            display("Результат деления исходного на проверка:")
            display(quo * buf_poly)
            display("-------------------------")
    if i >= factors_len:
        break

100%|██████████| 32/32 [00:03<00:00,  9.90it/s]


32


'Интерполяционный полином:'

'Результат деления исходного на интерполяционный:'

'Результат деления исходного на проверка:'

'-------------------------'